In [1]:
import torch
from torchviz import make_dot
from main_task_caption import init_model, DATALOADER_DICT

/home/karolwojtulewicz/miniconda3/envs/univl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wojaczek27. Use `wandb login --relogin` to force relogin


In [2]:
from main_task_caption import Args_Caption


args = Args_Caption(features_dir="data", do_eval=False, output_dir="Finetuned_models/ourds_nodiag_lang", export_attention_scores=False, task="caption-lang")
model = init_model(args, torch.device("cpu"), 0 , 0)

Stage-One:False, Stage-Two:True
Set bert_config.num_hidden_layers: 6.
Set visual_config.num_hidden_layers: 6.
Set bbx_config.num_hidden_layers: 6.
Set cross_config.num_hidden_layers: 6.
Set decoder_config.num_decoder_layers: 3.


768
30522


Weights from pretrained model cause errors in UniVL: 
   size mismatch for visual.embeddings.word_embeddings.weight: copying a param with shape torch.Size([768, 1024]) from checkpoint, the shape in current model is torch.Size([768, 768]).
   size mismatch for normalize_video.visual_norm2d.weight: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([768]).
   size mismatch for normalize_video.visual_norm2d.bias: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([768]).


In [3]:
import pickle
from modules.tokenization import BertTokenizer


tokenizer = BertTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case)
args.video_feature = pickle.load(open(args.features_path, 'rb'))
args.video_bbx_feature = pickle.load(open(args.bbx_features_path, 'rb'))

/home/karolwojtulewicz/code/NSVA/modules/bert-base-uncased/vocab.txt


In [4]:
val_dataloader, val_length = DATALOADER_DICT[args.datatype]["val"](args, tokenizer,split_type='val')

In [5]:
batch = next(iter(val_dataloader))
input_ids, input_mask, segment_ids, video, video_mask, \
pairs_masked_text, pairs_token_labels, masked_video, video_labels_index,\
pairs_input_caption_ids, pairs_decoder_mask, pairs_output_caption_ids,task_type, bbx, bbx_mask = batch

yhat = model(input_ids, segment_ids, input_mask, video.float(), video_mask.float(),
                pairs_masked_text=pairs_masked_text, pairs_token_labels=pairs_token_labels,
                masked_video=masked_video, video_labels_index=video_labels_index,
                input_caption_ids=pairs_input_caption_ids, decoder_mask=pairs_decoder_mask,
                output_caption_ids=pairs_output_caption_ids,task_type=task_type, bbx=bbx.float(), bbx_mask=bbx_mask.float())

In [6]:
make_dot(yhat, params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.410413 to fit


'rnn_torchviz.png'

In [7]:
import numpy as np
attn_mask = np.array([[[[    -0.,     -0.,     -0.,     -0.,     -0.,     -0.,     -0.,               -0.,     -0.,     -0.,     -0.,     -0.,     -0.,     -0.,               -0.,     -0., -10000., -10000., -10000., -10000., -10000.,           -10000., -10000., -10000., -10000., -10000., -10000., -10000.,           -10000., -10000.,     -0.,     -0.,     -0.,     -0.,     -0.,               -0.,     -0.,     -0.,     -0.,     -0.,     -0.,     -0.,               -0.,     -0.,     -0.,     -0., -10000., -10000., -10000.,           -10000., -10000., -10000., -10000., -10000., -10000., -10000.,           -10000., -10000., -10000., -10000.]]]])
attn_output = np.array([[[ 7.4819e-01, -1.7681e+00, -7.4810e-01,  6.6186e-01,
          -4.2375e-01, -1.5031e+00],
         [ 3.2637e-01, -1.2292e+00, -1.3774e+00,  3.8758e-01,
          -1.0174e+00, -3.6512e-01],
         [ 9.9983e-01, -1.8030e+00, -5.2177e-01,  4.7865e-02,
           1.3298e-03, -4.7057e-01],
         [-2.4457e-01,  9.7655e-03, -7.4755e-01,  2.1563e+00,
           1.7433e+00,  7.5925e-02],
         [-6.6065e-01, -9.9056e-01, -2.5342e-01,  1.2676e+00,
           1.1478e+00,  1.4212e+00],
         [ 1.4926e-01, -6.7882e-01, -1.2417e+00,  1.9228e+00,
           1.4427e+00,  1.1168e+00]]])

In [8]:
print(attn_mask.shape)
print(attn_output.shape)

(1, 1, 1, 60)
(1, 6, 6)
